In [1]:
from pathlib import Path
import pandas as pd
import ase.io
import ase.io.cif
import matbench_discovery.data

In [2]:
wbm_summary = pd.read_csv(matbench_discovery.data.DataFiles.wbm_summary.path, index_col='material_id')

In [3]:
wbm_atoms = matbench_discovery.data.ase_atoms_from_zip(
    matbench_discovery.data.DataFiles.wbm_relaxed_atoms.path, filename_to_info=True)

Reading ASE Atoms from zip_filename='/home/kna/.cache/matbench-discovery/wbm/2024-08-04-wbm-relaxed-atoms.extxyz.zip': 100%|██████████| 256963/256963 [01:03<00:00, 4015.45it/s]


In [4]:
structures = []
from io import BytesIO
for atoms in wbm_atoms:
    with BytesIO() as output:
        ase.io.cif.write_cif(output, atoms)
        structures.append({
            "material_id": atoms.info['material_id'],
            "cif": output.getvalue().decode()})

In [5]:
structures = pd.DataFrame.from_records(structures).set_index('material_id').reindex(wbm_summary.index)
structures['band_gap'] = wbm_summary['bandgap_pbe']
structures['formation_energy_per_atom'] = wbm_summary['e_form_per_atom_mp2020_corrected']

In [6]:
structures.to_csv(Path("..", "..", "data", "wbm", "test.csv.gz"), index_label=structures.index.name)